In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os

os.chdir("../")

# Plot of three monkey testers compared with the Approach

- Requires two identical C models with different random seeds
- Monkey Testers and Eval of C models need to have same amount of interactions
- Mean + Stddev over different runs of the same monkey tester
- Requires evaluation of C model (use `evaluate_controller.py` script)
- C model evaluations are first averaged, then these are again averaged + stddev is computed
- This then also plotted


In [ ]:
random_monkey_dir = "datasets/gui_env/random-clicks/2022-04-20_14-02-28"
random_widget_monkey_dir = "datasets/gui_env/random-widgets/2022-04-20_14-03-07"
random_widget_monkey_zero_random_click_prob_dir = "datasets/gui_env/random-widgets/2022-04-20_14-04-12"

first_controller_model_file = "controller_v_103_eval.npz"
second_controller_model_file = "controller_v_76_eval.npz"

In [ ]:
random_monkey_rewards = []
random_widget_monkey_rewards = []
random_widget_monkey_zero_random_click_prob_rewards = []

for i in range(3):
    
    if i == 0:
        monkey_dir = random_monkey_dir
        reward_list = random_monkey_rewards
    elif i == 1:
        monkey_dir = random_widget_monkey_dir
        reward_list = random_widget_monkey_rewards
    elif i == 2:
        monkey_dir = random_widget_monkey_zero_random_click_prob_dir
        reward_list = random_widget_monkey_zero_random_click_prob_rewards
    
    sequence_dirs = os.listdir(monkey_dir)
    sequence_dirs.sort()


    for seq_dir in sequence_dirs:
        sequence_data = np.load(os.path.join(monkey_dir, seq_dir, "data.npz"))
        reward_list.append(sequence_data["rewards"])
        
random_monkey_mean = np.mean(random_monkey_rewards, axis=0).cumsum()
random_monkey_std = np.std(random_monkey_rewards, axis=0)

random_widget_monkey_mean = np.mean(random_widget_monkey_rewards, axis=0).cumsum()
random_widget_monkey_std = np.std(random_widget_monkey_rewards, axis=0)

random_widget_monkey_zero_random_click_prob_mean = np.mean(random_widget_monkey_zero_random_click_prob_rewards, axis=0).cumsum()
random_widget_monkey_zero_random_click_prob_std = np.std(random_widget_monkey_zero_random_click_prob_rewards, axis=0)

In [ ]:
first_controller_rewards = []
second_controller_rewards = []


first_controller_rewards = np.load(first_controller_model_file)["all_rewards"]
second_controller_rewards = np.load(second_controller_model_file)["all_rewards"]

first_controller_mean = np.mean(first_controller_rewards, axis=0)
second_controller_mean = np.mean(second_controller_rewards, axis=0)

controller_mean = np.mean([first_controller_mean, second_controller_mean], axis=0).cumsum()
controller_std = np.std([first_controller_mean, second_controller_mean], axis=0)

In [ ]:
data_length_x = random_monkey_mean.shape

assert data_length_x == random_monkey_std.shape
assert data_length_x == random_widget_monkey_mean.shape
assert data_length_x == random_widget_monkey_std.shape
assert data_length_x == random_widget_monkey_zero_random_click_prob_mean.shape
assert data_length_x == random_widget_monkey_zero_random_click_prob_std.shape

assert data_length_x == controller_mean.shape
assert data_length_x == random_monkey_std.shape

fig, ax = plt.subplots()
x_data = np.arange(data_length_x[0])

ax.plot(x_data, random_monkey_mean, label="Random Monkey Tester")
ax.fill_between(x_data, random_monkey_mean - random_monkey_std, random_monkey_mean + random_monkey_std, alpha=0.25)

ax.plot(x_data, random_widget_monkey_mean, label="Random Widget Monkey Tester (0.125)")
ax.fill_between(x_data, random_widget_monkey_mean - random_widget_monkey_std, random_widget_monkey_mean + random_widget_monkey_std, alpha=0.25)

ax.plot(x_data, random_widget_monkey_zero_random_click_prob_mean, label="Random Widget Monkey Tester (0.0)")
ax.fill_between(x_data, random_widget_monkey_zero_random_click_prob_mean - random_widget_monkey_zero_random_click_prob_std,
                 random_widget_monkey_zero_random_click_prob_mean + random_widget_monkey_zero_random_click_prob_std,
                 alpha=0.25)

ax.plot(x_data, controller_mean, label="Investigated Approach (v_275, m_187)")
ax.fill_between(x_data, controller_mean - controller_std, controller_mean + controller_std, alpha=0.25)

ax.set(ylabel="Code Coverage (%)", xlabel='Time Steps')
ax.legend(loc='lower right')

fig.savefig("c_model_comparison_monkey_tester.pdf", bbox_inches='tight')
plt.show()